In [24]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
ret, ret_nonan, univ, tradeidx, dates = Util.load_data()

In [4]:
Ns = [100, 200, 500, 1000]
ratios = [0.25, 0.5, 1, 2, 4, None]
P = 1

In [31]:
for N in Ns:
    for r in ratios:
        if r is None:
            T = 1260
        else:
            T = int(r * N)
        if T >= 2000:
            continue
            #then the array will contain nans
        print("Working on N = {}, T = {}".format(N, T))
        folder = "historical_N_{}_T_{}".format(N, T)
        #Util.get_EWTQ_OOS(folder, N, P, univ, tradeidx, ret)
        #Util.get_MTP2_momentum_OOS(folder, N, P, univ, tradeidx, ret)
        #Util.get_AFM_momentum_OOS(folder, N, P, univ, tradeidx, ret)
        #Util.get_LS_momentum_OOS(folder, N, P, univ, tradeidx, ret)

Working on N = 100, T = 25
Working on N = 100, T = 50
Working on N = 100, T = 100
Working on N = 100, T = 200
Working on N = 100, T = 400
Working on N = 100, T = 1260
Working on N = 200, T = 50
Working on N = 200, T = 100
Working on N = 200, T = 200
Working on N = 200, T = 400
Working on N = 200, T = 800
Working on N = 200, T = 1260
Working on N = 500, T = 125
Working on N = 500, T = 250
Working on N = 500, T = 500
Working on N = 500, T = 1000
Working on N = 500, T = 1260
Working on N = 1000, T = 250
Working on N = 1000, T = 500
Working on N = 1000, T = 1000
Working on N = 1000, T = 1260


In [86]:
def initialize_df():
    N_t_tuples = []
    for N in Ns:
        for r in ratios:
            if r is None:
                T = 1260
            else:
                T = int(r * N)
            if T >= 2000:
                continue
            if Util.skip_weak:
                continue
            N_t_tuples.append((N, T))

    idx = pd.MultiIndex.from_tuples(N_t_tuples, names=["N\n(number of assets)", "T\n(lookback period)"])
    col = ['EW TQ', 'Linear Shrinkage', 'Approximate Factor Model', 'MTP2']
    df = pd.DataFrame('-', idx, col)
    return df

In [87]:
df = initialize_df()

In [88]:
for N in Ns:
    if N > 1000:
        continue
    for r in ratios:
        if r is None:
            T = 1260
        else:
            T = int(r * N)
        if T >= 2000:
            continue
        if Util.skip_weak(N, T):
            continue
        folder = "historical_N_{}_T_{}".format(N, T)
        MTP2_rets, LS_rets, AFM_rets = Util.load_OOS(folder, AFM = True,
                                                     momentum = True)
        EWTQ_rets = Util.load_EWTQ_OOS(folder)
        p_vals_LS = Util.read_hyp_results(folder, 'MTP2', 'LS', sharpe = True)
        p_vals_AFM = Util.read_hyp_results(folder, 'MTP2', 'AFM', sharpe = True)
        
        afm_p_val = p_vals_AFM.values[0][2]
        addendum = ""
        if afm_p_val <= 0.01:
            addendum += "***"
        elif afm_p_val <= 0.05:
            addendum += "**"
        elif afm_p_val <= 0.1:
            addendum += "*"
            
        df.at[(N, T), 'EW TQ'] = "{0:.3f}".format(Util.get_IR(EWTQ_rets)[2])
        df.at[(N, T), 'Linear Shrinkage'] = "{0:.3f}".format(Util.get_IR(LS_rets)[2])
        df.at[(N, T), 'MTP2'] = "{0:.3f}".format(Util.get_IR(MTP2_rets)[2]) + addendum
        df.at[(N, T), 'Approximate Factor Model'] = "{0:.3f}".format(Util.get_IR(AFM_rets)[2])
        #df.at[(N, T), 'vs LS p values'] = p_vals_LS.values[0]
        #df.at[(N, T), 'vs AFM p values'] = afm_p_val

In [89]:
df

EW TQ Linear Shrinkage  \
N\n(number of assets) T\n(lookback period)                           
100                   25                    0.694            0.710   
                      50                    0.694            0.625   
                      100                   0.694            0.600   
                      200                   0.694            0.670   
                      400                   0.694            0.736   
                      1260                  0.694            0.831   
200                   50                    0.757            0.742   
                      100                   0.757            0.719   
                      200                   0.757            0.812   
                      400                   0.757            0.864   
                      800                   0.757            0.967   
                      1260                  0.757            0.906   
500                   125                   0.764            0.876   
                      250                   0.764            0.985   
                      500                   0.764            0.940   
                      1000                  0.764            0.918   

                                           Approximate Factor Model     MTP2  
N\n(number of assets) T\n(lookback period)                                    
100                   25                                      0.730    0.803  
                      50                                      0.637    0.849  
                      100                                     0.617  0.896**  
                      200                                     0.688   0.899*  
                      400                                     0.782    0.892  
                      1260                                    0.834    0.890  
200                   50                                      0.726    0.853  
                      100                                     0.716    0.829  
                      200                                     0.800    0.885  
                      400                                     0.870    0.886  
                      800                                     0.961    0.970  
                      1260                                    0.916    0.955  
500                   125                                     0.872    1.019  
                      250                                     0.977    1.112  
                      500                                     0.980    1.045  
                      1000                                    0.978    1.061

In [90]:
print(df.to_latex())

\begin{tabular}{llllll}
\toprule
    &      &  EW TQ & Linear Shrinkage & Approximate Factor Model &     MTP2 \\
N
(number of assets) & T
(lookback period) &        &                  &                          &          \\
\midrule
100 & 25   &  0.694 &            0.710 &                    0.730 &    0.803 \\
    & 50   &  0.694 &            0.625 &                    0.637 &    0.849 \\
    & 100  &  0.694 &            0.600 &                    0.617 &  0.896** \\
    & 200  &  0.694 &            0.670 &                    0.688 &   0.899* \\
    & 400  &  0.694 &            0.736 &                    0.782 &    0.892 \\
    & 1260 &  0.694 &            0.831 &                    0.834 &    0.890 \\
200 & 50   &  0.757 &            0.742 &                    0.726 &    0.853 \\
    & 100  &  0.757 &            0.719 &                    0.716 &    0.829 \\
    & 200  &  0.757 &            0.812 &                    0.800 &    0.885 \\
    & 400  &  0.757 &            0.864 &      